In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@cc8cb8a

In [2]:
import spark.implicits._

case class Account(account_id: Int, income: Int)

val account: Seq[(Int, Int)] = Seq(
  (3, 108939),
  (2, 12747),
  (8, 87709),
  (6, 91796)
)

val accountsDS = account.toDF("account_id", "income").as[Account]

import spark.implicits._


defined class Account
account: Seq[(Int, Int)] = List((3, 108939), (2, 12747), (8, 87709), (6, 91796))
accountsDS: Dataset[Account] = [account_id: int, income: int]

Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:

* "Low Salary": All the salaries strictly less than $20000.
* "Average Salary": All the salaries in the inclusive range [$20000, $50000].
* "High Salary": All the salaries strictly greater than $50000.

The result table must contain all three categories. If there are no accounts in a category, return 0.

Return the result table in any order.

The result format is in the following example.

| category       | accounts_count |
|----------------|----------------|
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |

In [3]:
case class Salaries(category: String)
val salaries: Seq[(String)] = Seq(
    "Low Salary",
    "Average Salary",
    "High Salary",
)
val salariesDS = salaries.toDF("category").as[Salaries]

accountsDS.withColumn("cat", 
        when(
            col("income") < 20000, "Low Salary"
        ).when(
            (col("income") >= 20000 && col("income") <= 50000), "Average Salary"
        ).otherwise("High Salary")
    ).groupBy("cat")
    .agg(count(lit(1)).as("accounts_count"))
    .join(salariesDS, col("cat") === col("category"), "right")
    .select(
        col("category"),
        coalesce(col("accounts_count"), lit(0)).as("accounts_count")
    )
    .show()


+--------------+--------------+
|      category|accounts_count|
+--------------+--------------+
|    Low Salary|             1|
|Average Salary|             0|
|   High Salary|             3|
+--------------+--------------+



defined class Salaries
salaries: Seq[String] = List("Low Salary", "Average Salary", "High Salary")
salariesDS: Dataset[Salaries] = [category: string]

In [4]:
spark.stop()